In [ ]:
#This is a variant of "taketwo-dsmvp-v1-Feb9" in this and the "20newsgroup" directory for the Jigsaw data.

In [ ]:
#It is also a variant of the version that was pushed to the main branch of TakeTwo repo in Feb 21. 

In [1]:
#This is an attempt for the final version of "v1", eventually to work with the repo data

In [2]:
#This version contains both NB (Naive Bayes) and SGD (Support Vector Machine)

In [1]:
#In this version, the 20newsgroup data are being used for now. 

In [92]:
from sklearn.datasets import fetch_20newsgroups

In [93]:
from sklearn.feature_extraction.text import CountVectorizer

In [94]:
from sklearn.linear_model import SGDClassifier

In [95]:
from sklearn.naive_bayes import MultinomialNB

In [96]:
from sklearn.pipeline import Pipeline

In [97]:
from sklearn.feature_extraction.text import TfidfTransformer

In [98]:
from sklearn.model_selection import GridSearchCV

In [99]:
import numpy as np

In [100]:
import pandas as pd

In [177]:
import random

In [178]:
import string

In [101]:
#Fetch the Jigsaw data and store them in appropriate arrays

In [102]:
jigsawTrain = pd.read_csv('train.csv')

In [251]:
jigsawTrain.shape

(1804874, 45)

In [403]:
allTrain = pd.DataFrame(jigsawTrain, columns = ['id', 'comment_text', 'severe_toxicity'])

In [404]:
allTrain.shape

(1804874, 3)

In [378]:
allTrain1M = allTrain.head(1000000)

In [379]:
allTrain1M.shape

(1000000, 3)

In [380]:
#allTrain = allTrain1M

In [103]:
blackData = jigsawTrain[(jigsawTrain.black == 1.0)]

In [405]:
allTrain.shape

(1804874, 3)

In [104]:
blackTrain = pd.DataFrame(blackData, columns = ['id', 'comment_text', 'severe_toxicity', 'black'])

In [105]:
blackTrain.shape

(7656, 4)

In [106]:
blackTrainId = blackTrain['id']

In [107]:
blackTrainText = blackTrain['comment_text']

In [108]:
blackTrainSevereToxicity = blackTrain.loc[:,'severe_toxicity']

In [547]:
blackTrainText.head(1)

516    BJ, your suggestion that low income residents ...
Name: comment_text, dtype: object

In [406]:
allTrainId = allTrain['id']

In [407]:
allTrainText = allTrain['comment_text']

In [408]:
allTrainSevereToxicity = allTrain.loc[:,'severe_toxicity']

In [409]:
allTrainSevereToxicity.head()

0    0.000000
1    0.000000
2    0.000000
3    0.000000
4    0.021277
Name: severe_toxicity, dtype: float64

In [109]:
blackTrain.head(5)

,id,comment_text,severe_toxicity,black
516,240627,"BJ, your suggestion that low income residents ...",0.0,1.0
1428,242346,"Well, this wasn't meant to be a dissertation. ...",0.0,1.0
1527,242454,So Hillary's appeal is to Democrats who make o...,0.0,1.0
1569,242498,Is the real point of this article to show that...,0.0,1.0
1623,242562,> A less sensational headline would be “Black ...,0.0,1.0


In [410]:
allTrain.head(5)

,id,comment_text,severe_toxicity
0,59848,"This is so cool. It's like, 'would you want yo...",0.000000
1,59849,Thank you!! This would make my life a lot less...,0.000000
2,59852,This is such an urgent design problem; kudos t...,0.000000
3,59855,Is this something I'll be able to install on m...,0.000000
4,59856,haha you guys are a bunch of losers.,0.021277


In [110]:
blackTrainId.iloc[0]

240627

In [111]:
blackTrainText.iloc[0]

"BJ, your suggestion that low income residents of Portland do not understand their own needs and desires perplexes me.  It is a bit snobby- as if the poor need middle class whites to tell them what they should want. And, for blacks and latinos, home ownership[ is pretty much their only chance for a nest egg.  Unless they are union they don't really get the jobs with stock options and generous 401k matches.  The fact that redlining once existed doesn't make the handful of black residents around Alberta less attached to their homes.\n\nYou are struggling with he fact that the working class folks in Portland do not want to live the New Urbanist dream.  Look at the big study Metro did to gauge housing preferences. \n\nFinally, I note you put a lot of trust in the people who own large apartment complexes to not exploit the inherent vulnerability that comes with being a renter.  The SFH gives people autonomy.  Yes, housing prices can drop, but so can any other investment.  Take my 401k- plea

In [112]:
blackTrainSevereToxicity.iloc[0]

0.0

In [189]:
len(blackTrain)

7656

In [411]:
categories = ["non-toxic", "toxic"]

In [191]:
from collections import namedtuple
data_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [193]:
def flipCoin(bias):
    f = random.random()
    return True if f<bias else False

In [259]:
for i in range(0,10):
    print(flipCoin(0.25))

True
False
False
False
False
False
False
False
True
True


In [195]:
for x in range(len(blackTrain)):
    if (blackTrainSevereToxicity.iloc[x] > 0):
        data_set.target.append(1)
        data_set.urls.append(blackTrainId.iloc[x])
        data_set.data.append(blackTrainText.iloc[x])
    elif flipCoin(0.25):
        data_set.target.append(0)
        data_set.urls.append(blackTrainId.iloc[x])
        data_set.data.append(blackTrainText.iloc[x])

In [480]:
from collections import namedtuple
all_data_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [481]:
from collections import namedtuple
all_train_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [482]:
from collections import namedtuple
all_test_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [483]:
downSampling = 0.06

In [484]:
for x in range(len(allTrain)):
    if (allTrainSevereToxicity.iloc[x] > 0):
        all_data_set.target.append(1)
        all_data_set.urls.append(allTrainId.iloc[x])
        all_data_set.data.append(allTrainText.iloc[x])
    elif flipCoin(downSampling):
        all_data_set.target.append(0)
        all_data_set.urls.append(allTrainId.iloc[x])
        all_data_set.data.append(allTrainText.iloc[x])

In [485]:
#Here is a version in which training and test data are separately generated from all data

In [486]:
trainRatio = 0.9

In [487]:
for x in range(len(allTrain)):
    if (allTrainSevereToxicity.iloc[x] > 0):
        if flipCoin(trainRatio): 
            all_train_set.target.append(1)
            all_train_set.urls.append(allTrainId.iloc[x])
            all_train_set.data.append(allTrainText.iloc[x])
        else: 
            all_test_set.target.append(1)
            all_test_set.urls.append(allTrainId.iloc[x])
            all_test_set.data.append(allTrainText.iloc[x])
    elif flipCoin(downSampling):
        if flipCoin(trainRatio): 
            all_train_set.target.append(0)
            all_train_set.urls.append(allTrainId.iloc[x])
            all_train_set.data.append(allTrainText.iloc[x])
        else: 
            all_test_set.target.append(0)
            all_test_set.urls.append(allTrainId.iloc[x])
            all_test_set.data.append(allTrainText.iloc[x])

In [488]:
#either leave data_set as is (generated from blackData) or set it to all_data_set (generated from all the data)

In [493]:
data_set = all_test_set

In [494]:
len(data_set.target)

20810

In [495]:
 data_set.target[0:10]

[0, 0, 1, 1, 0, 0, 1, 1, 0, 1]

In [496]:
sum(data_set.target)

10597

In [428]:
data_set.urls[0:10]

[59856, 59859, 59861, 239583, 239607, 239612, 239655, 239711, 239733, 239762]

In [429]:
data_set.data[0:2]

['haha you guys are a bunch of losers.', 'ur a sh*tty comment.']

In [497]:
marker_train = all_train_set

In [498]:
#Here a separate test data set should be used.

In [499]:
marker_test = all_test_set

In [500]:
for t in marker_train.target[:10]:
...     print(marker_train.target_names[t])

toxic
toxic
toxic
non-toxic
toxic
toxic
non-toxic
non-toxic
toxic
non-toxic


In [501]:
#marker_train.data is an array of strings (containing as many "texts" as there are coming from Marker.)

In [502]:
#marker_train.target is an array of integers (containing the class indices for the above texts' class labels)

In [503]:
#marker_train_target_names is an array of class label names indexed by the array indices.

In [504]:
#flagger_new_text = fetch_20newsgroups(subset='test',
#                                      categories=categories, shuffle=True, random_state=42)

In [505]:
len(marker_train.target)

185473

In [506]:
sum(marker_train.target)

94039

In [507]:
len(marker_test.data)

20810

In [508]:
for t in marker_train.target[:10]:
...     print(marker_train.target_names[t])

toxic
toxic
toxic
non-toxic
toxic
toxic
non-toxic
non-toxic
toxic
non-toxic


In [509]:
for t in marker_test.target[:10]:
...     print(marker_train.target_names[t])

non-toxic
non-toxic
toxic
toxic
non-toxic
non-toxic
toxic
toxic
non-toxic
toxic


In [510]:
#Classes for model representations packaging the components in the modeling pipeline.
#For SGDClassifier and NaiveBayes.
#We may want to extend it to utilize "Pipeline()" as in "ModelRepSGD0", which is not in use at the moment.

In [511]:
class ModelRepSGD:
    def __init__(self, count_vect, X_train_counts, tf_transformer, text_clf):
        self.count_vect = CountVectorizer()
        self.X_train_counts = self.count_vect.fit_transform(marker_train.data)
        self.tf_transformer = TfidfTransformer(use_idf=False).fit(self.X_train_counts)
        self.text_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                  alpha=1e-3, random_state=42,
                                  max_iter=5, tol=None)),
        ])

In [512]:
class ModelRepNB:
    def __init__(self, count_vect, X_train_counts, tf_transformer, clf):
        self.count_vect = CountVectorizer()
        self.X_train_counts = self.count_vect.fit_transform(marker_train.data)
        self.tf_transformer = TfidfTransformer(use_idf=False).fit(self.X_train_counts)
        self.clf = MultinomialNB()

In [513]:
#Functions for model training - for SGDClassifier and NaiveBayes (SGD one is untested)

In [514]:
def model_train_SGD(marker_train): 
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(marker_train.data)
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
    X_train_tf = tf_transformer.transform(X_train_counts)
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',
                              alpha=1e-3, random_state=42,
                              max_iter=5, tol=None)),
    ])
    text_clf.fit(marker_train.data, marker_train.target)
    trained_model = ModelRepSGD(count_vect, X_train_counts, tf_transformer, text_clf)
    trained_model.text_clf = text_clf
    return trained_model

In [515]:
def model_train_NB(marker_train): 
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(marker_train.data)
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
    X_train_tf = tf_transformer.transform(X_train_counts)
    clf = MultinomialNB().fit(X_train_tf, marker_train.target)
    trained_model = ModelRepNB(count_vect, X_train_counts, tf_transformer, clf)
    trained_model.clf = clf
    return trained_model

In [516]:
#Function for model testing (common for SGDClassifier and NB)

In [517]:
def model_test(flagger_new_test, trained_model):
    count_vect = trained_model.count_vect
    tf_transformer = trained_model.tf_transformer
    clf = trained_model.clf
    X_new_counts = count_vect.transform(flagger_new_test)
    X_new_tfidf = tf_transformer.transform(X_new_counts)
    predicted = clf.predict(X_new_tfidf)
    return predicted

In [518]:
#Now testing model_train and model_test

In [519]:
trained_model_NB = model_train_NB(marker_train)

In [520]:
trained_model_SGD = model_train_SGD(marker_train)

In [521]:
#Inspecting the components of the trained model

In [522]:
trained_model_NB.clf

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [523]:
trained_model_NB.count_vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [524]:
trained_model_NB.tf_transformer

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=False)

In [525]:
trained_model_NB.clf

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [526]:
trained_model_SGD.count_vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [527]:
trained_model_SGD.tf_transformer

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=False)

In [528]:
trained_model_SGD.text_clf

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                      

In [529]:
trained_model_SGD.count_vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [530]:
trained_model_SGD.tf_transformer

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=False)

In [531]:
#Calling model tester/predictor on a single new text from Flagger

In [532]:
flagger_new_text = ['OpenGL on the GPU is fast']

In [533]:
predicted_NB = model_test(flagger_new_text, trained_model_NB)

In [534]:
predicted_SGD = trained_model_SGD.text_clf.predict(flagger_new_text)

In [535]:
predicted_NB

array([0])

In [536]:
predicted_SGD

array([0])

In [537]:
#Now evaluating NB and SGD models on a test data set

In [538]:
predicted_test_NB = model_test(marker_test.data, trained_model_NB)

In [539]:
predicted_test_SGD = trained_model_SGD.text_clf.predict(marker_test.data)

In [540]:
predicted_test_SGD.shape

(20810,)

In [541]:
sum(predicted_test_SGD)

9332

In [542]:
sum(predicted_test_NB)

11385

In [543]:
predicted_test_SGD[0:20]

array([0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [544]:
marker_test.target

[0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,


In [545]:
np.mean(predicted_test_NB == marker_test.target)

0.7641518500720808

In [546]:
np.mean(predicted_test_SGD == marker_test.target)

0.7134550696780394